# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san antonio de pale,-1.4014,5.6325,23.73,87,45,4.66,GQ,1693186452
1,1,okhotsk,59.3833,143.3000,21.72,52,14,5.41,RU,1693186452
2,2,villa azueta,18.1333,-95.6667,25.04,91,8,2.08,MX,1693186453
3,3,akureyri,65.6835,-18.0878,12.96,58,0,4.63,IS,1693186453
4,4,san patricio,28.0170,-97.5169,32.05,48,0,3.62,US,1693186454


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    size = "Humidity",
    scale = 2,
    alpha = 0.3,
    geo = True,
    tiles = "OSM",
    frame_width = 1500,
    frame_height = 1000)

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27
min_temp = 21
max_wind = 4.5
cloud_cover = 20
min_humid = 20
max_humid = 80

# Drop any rows with null values
clean_data_df = city_data_df.dropna(how="any")

# Given the base values provided above, define the ideal weather conditions
criteria_temp = (clean_data_df['Max Temp'] < max_temp) & (clean_data_df['Max Temp'] > min_temp)
criteria_wind = clean_data_df['Wind Speed'] < max_wind
criteria_cloud = clean_data_df['Cloudiness'] < cloud_cover
criteria_humid = (clean_data_df['Humidity'] < max_humid) & (clean_data_df['Humidity'] > min_humid)

# Create the dataframe with ideal weather conditions
ideal_weather_df = clean_data_df.loc[criteria_temp & criteria_wind & criteria_cloud & criteria_humid]

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,victoria point,-27.5833,153.3000,24.89,52,9,2.68,AU,1693186456
32,32,vilyuysk,63.7553,121.6247,23.81,35,3,2.28,RU,1693186464
52,52,pacific grove,36.6177,-121.9166,24.34,75,0,3.60,US,1693186472
65,65,avarua,-21.2078,-159.7750,24.03,60,17,3.09,CK,1693186477
83,83,mokhsogollokh,61.4681,128.9203,24.31,41,7,2.58,RU,1693186484


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(len(hotel_df.columns), 'Hotel Name', "")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,victoria point,AU,-27.5833,153.3000,52,
32,vilyuysk,RU,63.7553,121.6247,35,
52,pacific grove,US,36.6177,-121.9166,75,
65,avarua,CK,-21.2078,-159.7750,60,
83,mokhsogollokh,RU,61.4681,128.9203,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
victoria point - nearest hotel: Seminara Appartments
vilyuysk - nearest hotel: Мария
pacific grove - nearest hotel: Pacific Grove Inn
avarua - nearest hotel: Paradise Inn
mokhsogollokh - nearest hotel: No hotel found
bundaberg - nearest hotel: Young Australian Hotel
kasongo-lunda - nearest hotel: No hotel found
shitanjing - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
ponto novo - nearest hotel: No hotel found
khamis mushait - nearest hotel: No hotel found
morinville - nearest hotel: No hotel found
chaparral - nearest hotel: Europa
bodaybo - nearest hotel: НОЙ
argo - nearest hotel: Apollon Hotel
ruidoso - nearest hotel: Hotel Ruidoso
wahpeton - nearest hotel: Select Inn Breckenridge
prince rupert - nearest hotel: Crest Hotel
uturoa - nearest hotel: Hawaiki Nui hotel
amarante do maranhao - nearest hotel: No hotel found
essaouira - nearest hotel: Essaouira Wind Palace
dalandzadgad - nearest hotel: Хан Уул буудал
ketchikan - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
11,victoria point,AU,-27.5833,153.3000,52,Seminara Appartments
32,vilyuysk,RU,63.7553,121.6247,35,Мария
52,pacific grove,US,36.6177,-121.9166,75,Pacific Grove Inn
65,avarua,CK,-21.2078,-159.7750,60,Paradise Inn
83,mokhsogollokh,RU,61.4681,128.9203,41,No hotel found
97,bundaberg,AU,-24.8500,152.3500,56,Young Australian Hotel
109,kasongo-lunda,CD,-6.4667,16.8167,54,No hotel found
121,shitanjing,CN,39.2342,106.3439,39,No hotel found
140,carnarvon,AU,-24.8667,113.6333,43,No hotel found
149,ponto novo,BR,-10.8628,-40.1336,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Remove countries with 'No hotel found'
hotels_only_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == 'No hotel found'].index)

# Configure the map plot
hotel_map = hotels_only_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    size = "Humidity",
    scale = 8,
    alpha = 0.3,
    geo = True,
    tiles = "OSM",
    frame_width = 1500,
    frame_height = 1000,
    hover_cols = ['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)